In [28]:
# dosya okuma
file_path = "credit.data"
f = open(file_path, "r")
lines = f.read().splitlines()
num_row = len(lines) - 1
for line in lines:
    if line.startswith('#'):
        num_col = len(line[1:].split('\t'))
        break
num_feat = num_col - 1

In [29]:
# kullanılacak veri yapılarının tanımlanması
data = [] # veri tablosu
title = [] # başlıklar
column = [] # özniteliklerin aldığı değer kümeleri
sets = [set() for i in range(num_col)] # kümeler için geçici değişken

In [30]:
# veri tablosunun oluşturulması
for line in lines:
    if line.startswith('#'):
        title = line[1:].split('\t')
        continue
    values = line.split('\t')
    data.append(values)
    for i in range(num_col):
        sets[i].add(values[i])

In [31]:
# özniteliklerin aldığı değer kümelerinin oluşturulması
num_val = 0
for a_set in sets:
    column.append(sorted(list(a_set))) # her bir feature için elde edilen değerler kümesi listeye dönüştürülür
    num_val = num_val + len(a_set)
num_feat_val = num_val - len(column[-1])
num_label = len(column[-1])

In [32]:
print("column >> ", column)
print("number of values >> ", num_val)
print("number of feature values >> ", num_feat_val)

column >>  [['average', 'high', 'low'], ['middle', 'old', 'young'], ['debtless', 'indebted'], ['no', 'yes']]
number of values >>  10
number of feature values >>  8


In [33]:
freq_feat_val = [None] * num_feat # sınıflara göre öznitelik değer frekansları
freq_cls = [0] * num_label # sınıf etiket frekansları
for i in range(num_feat):
    freq_feat_val[i] = [[0 for j in range(num_label)] for k in range(len(column[i]))]

In [34]:
for row in data:
    for i, value in enumerate(row[:-1]):
        freq_feat_val[i][column[i].index(value)][column[-1].index(row[-1])] += 1
    freq_cls[column[-1].index(row[-1])] += 1
print("feature value frequencies by classes >> ", freq_feat_val)
print("class label frequencies >> ", freq_cls)

feature value frequencies by classes >>  [[[1, 4], [0, 5], [4, 1]], [[1, 4], [3, 2], [1, 4]], [[1, 7], [4, 3]]]
class label frequencies >>  [5, 10]


In [35]:
# Laplace (add-1) smoothing
found = False
for i in range(num_feat):
    for j in range(len(column[i])):
        for k in range(num_label):
            if freq_feat_val[i][j][k] == 0:
                found = True
                break
if (found):
    for i in range(num_feat):
        for j in range(len(column[i])):
            for k in range(num_label):
                freq_feat_val[i][j][k] += 1
print("feature value frequencies by classes >> ", freq_feat_val)

feature value frequencies by classes >>  [[[2, 5], [1, 6], [5, 2]], [[2, 5], [4, 3], [2, 5]], [[2, 8], [5, 4]]]


In [36]:
# sınıf etiketlerine göre öznitelik değer koşullu olasılıklarının hesaplanması
prob_feat = [None] * num_feat
for i in range(num_feat):
    prob_feat[i] = [[0 for j in range(num_label)] for k in range(len(column[i]))]
for i in range(num_feat):
    for j in range(len(column[i])):
        for k in range(num_label):
            prob_feat[i][j][k] = freq_feat_val[i][j][k] / freq_cls[k]
print("conditional probabilities p(x|c) >> ", prob_feat)

conditional probabilities p(x|c) >>  [[[0.4, 0.5], [0.2, 0.6], [1.0, 0.2]], [[0.4, 0.5], [0.8, 0.3], [0.4, 0.5]], [[0.4, 0.8], [1.0, 0.4]]]


In [37]:
# verilen bir girdi öznitelik vektörü için soncul olasılığı maksimum olan sınıf etiketini döndüren fonksiyon
def predict(input):
    post_prob = [1.0] * num_label
    for j in range(num_label):
        for i, value in enumerate(input):
            post_prob[j] *= prob_feat[i][column[i].index(value)][j]
    max = 0
    for i in range(len(post_prob)):
        if post_prob[i] > post_prob[max]:
            max = i;
    return column[-1][max]

In [38]:
# tekil test
input = ['average', 'middle', 'indebted']
print(predict(input))

no


In [39]:
# eğitim verisi üzerinde test
c = 0
for row in data:
    if predict(row[:-1]) == row[-1]:
        c += 1
print("accuracy >> ", c / len(data))

accuracy >>  0.8
